# Testing cVAE as harmonization tool on real brain data

In [32]:
import os.path
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
from mecvae.lit import RealBrainMeasuresDataModule

data_module = RealBrainMeasuresDataModule('/Users/ssilvari/Downloads/fedcombat_real_data/unified/non_harmonized_data.csv', batch_size=6)

In [34]:
data_module.prepare_data()
data_module.setup()

Reading /Users/ssilvari/Downloads/fedcombat_real_data/unified/non_harmonized_data.csv...
Number of numerical variables: self.n_num_cols=86
Number of categorical variables: self.n_cat_cols=2
Number of batches: 9


In [35]:
data_module.X

,C(Sex)[Female],C(Sex)[Male],C(DX)[T.Autism],C(DX)[T.CN],C(DX)[T.GenCohort PD],C(DX)[T.GenCohort Unaff],C(DX)[T.MCI],C(DX)[T.PD],C(DX)[T.Prodromal],C(DX)[T.SWEDD],...,"standardize(Q(""rh_cuneus_thickness""))","standardize(Q(""lh_parsopercularis_thickness""))","standardize(Q(""rh_postcentral_thickness""))","standardize(Q(""lh_precuneus_thickness""))","standardize(Q(""rh_caudalanteriorcingulate_thickness""))","standardize(Q(""rh_superiorparietal_thickness""))","standardize(Q(""lh_parahippocampal_thickness""))","standardize(Q(""rh_superiortemporal_thickness""))","standardize(Q(""lh_paracentral_thickness""))","standardize(Q(""lh_inferiorparietal_thickness""))"
PTID,,,,,,,,,,,,,,,,,,,,,
A4_B10081264,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.625918,0.289726,-0.697299,-0.328659,0.794624,-0.233408,0.593521,0.698243,0.327895,-0.624250
A4_B10102783,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.367826,-0.466325,-0.338380,-0.589114,-1.360042,-0.359543,1.105863,-0.322887,-0.696486,-0.095211
A4_B10108368,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.126208,-0.255220,0.416958,0.863949,-0.634536,0.422495,0.210886,0.366270,0.866129,-0.462202
A4_B10117310,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.466670,-0.692158,0.009826,-0.200716,-0.975436,-0.601723,0.807538,-0.398527,-0.783298,-0.895918
A4_B10350512,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.021873,-0.554695,-0.343737,-0.246410,0.510540,0.018862,-1.569341,-0.339696,-0.609674,-0.228662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PPMI_85062,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,-2.218398,-1.124188,-1.624062,-1.087176,-0.647647,-1.252580,0.110363,-0.385920,-3.062112,-0.814894
PPMI_85242,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,-0.109734,-0.215945,-0.633015,-0.054496,-0.949213,-0.273771,-1.105639,-0.268259,-0.366601,-0.381178
PPMI_90456,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,-0.181121,-0.461416,0.240177,0.164834,1.375903,1.386167,-1.245074,-0.726297,0.397344,1.239301


In [36]:
from mecvae.lit_models.cvae import LitFlexCVAE

# Import LitCVAE

model = LitFlexCVAE(data_dim=data_module.n_features,
                    conditioning_dim=data_module.n_batches,
                    # lr=1e-5, activation=nn.Tanh(),
                    hidden_dim=[256, 128],
                    z_dim=64,
                    optimizer='adam')
# Test model with batch
x, y = next(iter(data_module.train_dataloader()))
print(x.shape, y.shape)
x_hat, mu, log_var = model(x, y)

torch.Size([6, 96]) torch.Size([6, 9])


In [37]:
# Train model using pytorch lightning
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

# Define logger
logger = TensorBoardLogger(save_dir='/Users/ssilvari/PycharmProjects/Fed-MECVAE/lightning_logs', name='cVAE (Flex)')

# Save best model using a model checkpoint callback
checkpoint_callback = ModelCheckpoint(
    monitor='Validation/loss',
    dirpath=logger.log_dir,
    filename='cVAE (Flex)-{epoch:02d}-{Validation-loss:.2f}',
    save_top_k=1,
    mode='min',
)

# Define early stopping callback
callbacks = [EarlyStopping(monitor='Validation/loss', patience=10), checkpoint_callback]

# Define trainer
trainer = Trainer(max_epochs=2000, callbacks=callbacks, logger=logger, enable_progress_bar=False, accelerator='cpu',
                  gradient_clip_val=1.0, accumulate_grad_batches=4, precision=16)
# Continue from checkpoint
ckpt_path = checkpoint_callback.best_model_path if os.path.exists(checkpoint_callback.best_model_path) else None

# Train model
trainer.fit(model, data_module, ckpt_path=ckpt_path)

# Load best model
model = LitFlexCVAE.load_from_checkpoint(checkpoint_callback.best_model_path)

/Users/ssilvari/Library/Caches/pypoetry/virtualenvs/mecvae-_b3Dgs6W-py3.11/lib/python3.11/site-packages/lightning_fabric/connector.py:562: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
/Users/ssilvari/Library/Caches/pypoetry/virtualenvs/mecvae-_b3Dgs6W-py3.11/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:517: UserWarning: You passed `Trainer(accelerator='cpu', precision='16-mixed')` but AMP with fp16 is not supported on CPU. Using `precision='bf16-mixed'` instead.
  rank_zero_warn(
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Reading /Users/ssilvari/Downloads/fedcombat_real_data/unified/non_harmonized_data.csv...
Number of numerical variables: self.n_num_cols=86
Number of categorical variables: self.n_cat_cols=2


/Users/ssilvari/Library/Caches/pypoetry/virtualenvs/mecvae-_b3Dgs6W-py3.11/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(

  | Name      | Type       | Params
-----------------------------------------
0 | encoder   | Sequential | 60.0 K
1 | fc_mean   | Linear     | 8.3 K 
2 | fc_logvar | Linear     | 8.3 K 
3 | decoder   | Sequential | 67.2 K
-----------------------------------------
143 K     Trainable params
0         Non-trainable params
143 K     Total params
0.575     Total estimated model params size (MB)


Number of batches: 9


/Users/ssilvari/Library/Caches/pypoetry/virtualenvs/mecvae-_b3Dgs6W-py3.11/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/ssilvari/Library/Caches/pypoetry/virtualenvs/mecvae-_b3Dgs6W-py3.11/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/ssilvari/Library/Caches/pypoetry/virtualenvs/mecvae-_b3Dgs6W-py3.11/lib/python3.11/site-p

In [38]:
checkpoint_callback.best_model_path

'/Users/ssilvari/PycharmProjects/Fed-MECVAE/lightning_logs/cVAE (Flex)/version_4/cVAE (Flex)-epoch=01-Validation-loss=0.00.ckpt'

In [39]:
# Predict the whole dataset removing the batch effect by sampling y from a categorical distribution
import torch

# Compute probabilities of y
y_probs = data_module.y.value_counts(normalize=True).sort_index().values
y_probs = torch.tensor(y_probs, dtype=torch.float32)

# Extract tensors from data_module
x = torch.tensor(data_module.X.values, dtype=torch.float32)

# Create a list of predictions (we'll sample 100 times)
x_hats = []
for _ in range(100):
    with torch.no_grad():
        # Sample y
        y = torch.multinomial(y_probs, len(data_module.y), replacement=True)
        # One hot encode
        y = torch.nn.functional.one_hot(y, num_classes=len(y_probs)) #* 0
        # print(x.shape, y.shape)

        # Predict x
        x_hat, mu, log_var = model(x, y)

        # Append to list
        x_hats.append(x_hat.detach())

# Compute x_hat mean from samples
x_hat_mean = torch.stack(x_hats).mean(dim=0)

KeyboardInterrupt: 

In [ ]:
# Create a dataframe with the predictions
import os
import re
import pandas as pd

x_hat_df = pd.DataFrame(x_hat_mean.numpy(), columns=data_module.X.columns, index=data_module.df.index)

# Remove covariate columns. Those containing 'Age', 'Sex', 'DX'
x_hat_df = x_hat_df.loc[:, ~x_hat_df.columns.str.contains('Age|Sex|DX')]

# Extract the phenotype name using a regex knwong that the column names are wrapped by something like this: standardize(Q("lh_inferiorparietal_thickness")) where lh_inferiorparietal_thickness is the phenotype name
x_hat_df.columns = [re.search(r'Q\("(.*)"\)', col).group(1) for col in x_hat_df.columns]

# Back transform from the standardization
x_hat_df_destd = x_hat_df * data_module.df[x_hat_df.columns].std() + data_module.df[x_hat_df.columns].mean()
print(x_hat_df_destd.head())

# Join the rest of the columns present in data_module.df that are not in x_hat_df_destd
x_hat_df_destd = x_hat_df_destd.join(data_module.df.loc[:, ~data_module.df.columns.isin(x_hat_df_destd.columns)])
print(x_hat_df_destd)

# save as csv
root_dir = os.path.dirname(data_module.csv_file)
harmonized_csv = root_dir + os.sep + 'harmonized_cVAE.csv'
x_hat_df_destd.to_csv(harmonized_csv)
print(f'Harmonized data saved to {harmonized_csv}')

In [ ]:
# Add to `methods_params.csv` in the same folder
methods_params_file = root_dir + os.sep + 'methods_params.csv'
methods_params = pd.read_csv(methods_params_file)

# Create new entry
cvae_mparams = pd.DataFrame([{
    'Method': 'cVAE',
    'data_file_path': harmonized_csv,
    'classification_results_path': root_dir + os.sep + 'benchmark cVAE.csv'
}])

methods_params = pd.concat([methods_params, cvae_mparams], ignore_index=True).drop_duplicates()

# save as csv ignore index
methods_params.to_csv(methods_params_file, index=False)

In [ ]:
# Notify that the process is done
import os
os.system('afplay /System/Library/Sounds/Funk.aiff')

# Print date time when finished
import datetime
print(f'Finished at {datetime.datetime.now()}')